In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
import math, statistics
import random

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [36]:
random.seed(2)
tf.random.set_seed(2)

In [37]:
df_unstructure_full = pd.read_csv("/content/drive/MyDrive/코드스테이츠 과제/AFI_project/unstructed_data.csv")

In [38]:
df_unstructure_full

gamer_id  ...  y
0       000010f658e9f47d01fd060a769e1d95395ac7c97c357d...  ...  0
1       000012d4f585a9bb5d6975e157b79b3538b04885ac19c6...  ...  1
2       00002a907f341c6113a96699f6da7d69bff34f29853c47...  ...  1
3       00003fb64d5509abbd5921dfd4736d9fc8d2c392a42c25...  ...  1
4       0000bbd7d65a8cc77af85023344fb46e3a00d3772dcbf3...  ...  0
...                                                   ...  ... ..
545907  fffeb0212f0f68f342421974ddf60b60b4468ca71f89fa...  ...  1
545908  ffff55ab88728761706a92ef75d68394c5dad8edd66818...  ...  1
545909  ffff84a159d96864ec6ee5ae6c9c4386d3060666bcc5a1...  ...  1
545910  ffffebdf9cff7c9d118dbb90fe145fbeb61c4978e1692d...  ...  1
545911  fffff1f88815e0ba807c3ff28ebcb94d9dbd8fce805eae...  ...  0

[545912 rows x 3 columns]

# AutoEncoder + CNN

#### AutoEncoder

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from datetime import datetime
import math, statistics
import random

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

#####AutoEncoder(64) + CNN

In [ ]:
df_unstructure_full = pd.read_csv("/content/drive/MyDrive/코드스테이츠 과제/AFI_project/unstructed_data.csv")

In [ ]:
# 문장별 단어 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_unstructure_full['data'])

In [ ]:
# sample
print('총 샘플의 개수 : {}'.format(len(df_unstructure_full['data'])))

# vocab size 계산
vocab_size = len(tokenizer.word_index) + 1
print('vocab size : {}'.format(vocab_size))

# 정수 인코딩
X_encoded = tokenizer.texts_to_sequences(df_unstructure_full['data'])
print('encoded text example : {}'.format(X_encoded[0]))

# max length 계산
max_len = max(len(i) for i in X_encoded)
print('max length : {}'.format(max_len))

In [ ]:
# padding
X = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y = np.array(df_unstructure_full['y'])

In [ ]:
#  데이터분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

In [31]:
#64 임베딩
# 0. 사용할 패키지 불러오기
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D

max_features = 31951
text_max_words = 85

encoder = Sequential()
encoder.add(Embedding(max_features, 128, input_length=text_max_words))
encoder.add(Dropout(0.2))
encoder.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
encoder.add(GlobalMaxPooling1D())
encoder.add(Dense(128, activation='relu'))
encoder.add(Dropout(0.2))
encoder.add(Dense(64,activation='softmax'))

decoder = Sequential()
decoder.add(keras.layers.Dense(64, input_shape=[64]))
decoder.add(Dropout(0.2))
decoder.add(Dense(85, activation='softmax'))

autoencoder = keras.models.Sequential([encoder,decoder])

# 3. 모델 학습과정 설정하기
autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
hist = autoencoder.fit(X_train, X_train, epochs=1, batch_size=64, validation_data=(X_test, X_test))

6824/6824 [==============================] - 94s 14ms/step - loss: -172494.7656 - accuracy: 0.0041 - val_loss: -475614.0312 - val_accuracy: 0.0000e+00


In [ ]:
encoder.load_weights('/content/drive/MyDrive/코드스테이츠 과제/AFI_project/weight_256.ckpt')
prediction = encoder.predict(X_train)
prediction.shape

In [32]:
prediction = encoder.predict(X_train)
prediction.shape

(436729, 64)

In [33]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.3
hidden_dim = 128
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2


model64_full = Sequential()
Embed = Embedding(436729, 64, weights=[prediction], input_length=max_len, trainable=False)
model64_full.add(Embed)
model64_full.add(Conv1D(num_filters, kernel_size=kernel_size, padding='valid', activation='relu'))
model64_full.add(GlobalMaxPooling1D())
model64_full.add(Dropout(dropout_rate))
model64_full.add(Dense(units=hidden_dim, activation='relu'))
model64_full.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [34]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model64_full.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [35]:
history64_full = model64_full.fit(X_train, y_train,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=VALID_SPLIT,
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/100
683/683 [==============================] - 10s 14ms/step - loss: 0.6809 - acc: 0.5789 - val_loss: 0.6804 - val_acc: 0.5801
Epoch 2/100
683/683 [==============================] - 9s 13ms/step - loss: 0.6807 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 3/100
683/683 [==============================] - 8s 12ms/step - loss: 0.6807 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 4/100
683/683 [==============================] - 9s 13ms/step - loss: 0.6807 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 5/100
683/683 [==============================] - 8s 12ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 6/100
683/683 [==============================] - 8s 12ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6804 - val_acc: 0.5801
Epoch 7/100
683/683 [==============================] - 9s 13ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 8/100
683/683 [==============================] - 9s 13m

In [36]:
# 확률값
y_pred64 = model64_full.predict(X_test)
# 클래스
y_pred_class64 = np.around(y_pred64)

In [37]:
report = classification_report(y_test, y_pred_class64)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     46122
           1       0.58      1.00      0.73     63061

    accuracy                           0.58    109183
   macro avg       0.29      0.50      0.37    109183
weighted avg       0.33      0.58      0.42    109183



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#####AutoEncoder(128) + CNN

In [ ]:
df_unstructure_full = pd.read_csv("/content/drive/MyDrive/코드스테이츠 과제/AFI_project/unstructed_data.csv")

In [ ]:
# 문장별 단어 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_unstructure_full['data'])

In [ ]:
# sample
print('총 샘플의 개수 : {}'.format(len(df_unstructure_full['data'])))

# vocab size 계산
vocab_size = len(tokenizer.word_index) + 1
print('vocab size : {}'.format(vocab_size))

# 정수 인코딩
X_encoded = tokenizer.texts_to_sequences(df_unstructure_full['data'])
print('encoded text example : {}'.format(X_encoded[0]))

# max length 계산
max_len = max(len(i) for i in X_encoded)
print('max length : {}'.format(max_len))

In [ ]:
# padding
X = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y = np.array(df_unstructure_full['y'])

In [ ]:
#  데이터분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

In [ ]:
#128임베딩
# 0. 사용할 패키지 불러오기
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D, Conv1DTranspose
import tensorflow as tf
from tensorflow import keras

max_features = 31951
text_max_words = 85

encoder = Sequential()
encoder.add(Embedding(max_features, 256, input_length=text_max_words))
encoder.add(Dropout(0.2))
encoder.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
encoder.add(GlobalMaxPooling1D())
encoder.add(Dense(128, activation='relu'))

decoder = Sequential()
decoder.add(keras.layers.Dense(128, input_shape=[128]))
decoder.add(Dropout(0.2))
decoder.add(Dense(85, activation='softmax'))

autoencoder = keras.models.Sequential([encoder,decoder])

# 3. 모델 학습과정 설정하기
autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
autoencoder.summary()
# 4. 모델 학습시키기
#hist = autoencoder.fit(X_train, X_train, epochs=1, batch_size=64, validation_data=(X_test, X_test))

In [ ]:
encoder.load_weights('/content/drive/MyDrive/코드스테이츠 과제/AFI_project/weight_128.ckpt')

In [ ]:
prediction = encoder.predict(X_train)
prediction.shape

In [ ]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.3
hidden_dim = 128
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2


model128_full = Sequential()
Embed = Embedding(436729, 128, weights=[prediction], input_length=max_len, trainable=False)
model128_full.add(Embed)
model128_full.add(Conv1D(num_filters, kernel_size=kernel_size, padding='valid', activation='relu'))
model128_full.add(GlobalMaxPooling1D())
model128_full.add(Dropout(dropout_rate))
model128_full.add(Dense(units=hidden_dim, activation='relu'))
model128_full.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model128_full.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

In [ ]:
history128_full = model128_full.fit(X_train, y_train,
                       epochs=NUM_EPOCHS,
                       batch_size=BATCH_SIZE,
                       validation_split=VALID_SPLIT,
                       verbose=1,
                       callbacks=[early_stopping])

In [ ]:
# 확률값
y_pred128 = model128_full.predict(X_test)
# 클래스
y_pred_class128 = np.around(y_pred128)

#####AutoEncoder(256) + CNN

In [7]:
df_unstructure_full = pd.read_csv("/content/drive/MyDrive/코드스테이츠 과제/AFI_project/unstructed_data.csv")

In [10]:
# 문장별 단어 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_unstructure_full['data'])

In [11]:
# sample
print('총 샘플의 개수 : {}'.format(len(df_unstructure_full['data'])))

# vocab size 계산
vocab_size = len(tokenizer.word_index) + 1
print('vocab size : {}'.format(vocab_size))

# 정수 인코딩
X_encoded = tokenizer.texts_to_sequences(df_unstructure_full['data'])
print('encoded text example : {}'.format(X_encoded[0]))

# max length 계산
max_len = max(len(i) for i in X_encoded)
print('max length : {}'.format(max_len))

총 샘플의 개수 : 545912
vocab size : 31951
encoded text example : [2, 334, 1, 84, 1, 18, 21, 5, 16, 43, 126, 433, 350, 31, 65, 14, 60, 219, 734, 791, 51, 104, 13, 20, 90, 327, 135, 36, 69, 10, 20, 90, 249, 135, 36, 69]
max length : 85


In [12]:
# padding
X = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y = np.array(df_unstructure_full['y'])

In [13]:
#  데이터분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2, random_state=42)

In [16]:
#256임베딩
# 0. 사용할 패키지 불러오기
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, GlobalMaxPooling1D, Conv1DTranspose
import tensorflow as tf
from tensorflow import keras

max_features = 31951
text_max_words = 85

encoder = Sequential()
encoder.add(Embedding(max_features, 512, input_length=text_max_words))
encoder.add(Dropout(0.2))
encoder.add(Conv1D(512,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
encoder.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
encoder.add(GlobalMaxPooling1D())
encoder.add(Dense(256, activation='relu'))
# encoder.add(Dropout(0.2))
# encoder.add(Dense(64))

decoder = Sequential()
decoder.add(keras.layers.Dense(256, input_shape=[256]))
decoder.add(Dropout(0.2))
decoder.add(Dense(85, activation='softmax'))

autoencoder = keras.models.Sequential([encoder,decoder])

# 3. 모델 학습과정 설정하기
autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
autoencoder.summary()
# 4. 모델 학습시키기
#hist = autoencoder.fit(X_train, X_train, epochs=1, batch_size=64, validation_data=(X_test, X_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 256)               17605120  
                                                                 
 sequential_5 (Sequential)   (None, 85)                87637     
                                                                 
Total params: 17,692,757
Trainable params: 17,692,757
Non-trainable params: 0
_________________________________________________________________


In [17]:
encoder.load_weights('/content/drive/MyDrive/코드스테이츠 과제/AFI_project/weight_256.ckpt')
prediction = encoder.predict(X_train)
prediction.shape

(436729, 256)

In [19]:
num_filters = 100
kernel_size = 3
dropout_rate = 0.5
hidden_dim = 32
output_dim = 1
BATCH_SIZE = 512
NUM_EPOCHS = 100
VALID_SPLIT = 0.2
vocab_size = 436729


model256 = Sequential()
Embed = Embedding(vocab_size, 256, weights=[prediction], input_length=max_len, trainable=False)
model256.add(Embed)
model256.add(Conv1D(128, kernel_size=3, padding='valid', activation='relu'))
model256.add(Conv1D(64, kernel_size=3, padding='valid', activation='relu'))
model256.add(Conv1D(32, kernel_size=3, padding='valid', activation='relu'))
model256.add(GlobalMaxPooling1D())
model256.add(Dropout(dropout_rate))
model256.add(Dense(units=hidden_dim, activation='relu'))
model256.add(Dense(units=16, activation='relu'))
model256.add(Dense(units=1, activation='sigmoid')) # 0.0~1.0 실수로 출력

In [21]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model256.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, verbose=0, mode='auto')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
history256 = model256.fit(X_train, y_train,
                       epochs=NUM_EPOCHS,
                       batch_size=BATCH_SIZE,
                       validation_split=VALID_SPLIT,
                       verbose=1,
                       callbacks=[early_stopping])

Epoch 1/100
683/683 [==============================] - 27s 36ms/step - loss: 12944475136.0000 - acc: 0.5653 - val_loss: 14227.6885 - val_acc: 0.5801
Epoch 2/100
683/683 [==============================] - 23s 34ms/step - loss: 15661.2959 - acc: 0.5791 - val_loss: 9624.3252 - val_acc: 0.5801
Epoch 3/100
683/683 [==============================] - 23s 34ms/step - loss: 9614.7754 - acc: 0.5791 - val_loss: 4272.4146 - val_acc: 0.5801
Epoch 4/100
683/683 [==============================] - 23s 34ms/step - loss: 2551.0249 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 5/100
683/683 [==============================] - 23s 34ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 6/100
683/683 [==============================] - 23s 34ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 7/100
683/683 [==============================] - 23s 34ms/step - loss: 0.6806 - acc: 0.5791 - val_loss: 0.6803 - val_acc: 0.5801
Epoch 8/100
683/683 [====

In [23]:
# 확률값
y_pred256 = model256.predict(X_test)

In [24]:
# 클래스
y_pred_class256 = np.around(y_pred256)

In [26]:
report = classification_report(y_test, y_pred_class256)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     46122
           1       0.58      1.00      0.73     63061

    accuracy                           0.58    109183
   macro avg       0.29      0.50      0.37    109183
weighted avg       0.33      0.58      0.42    109183



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Classification Report

In [33]:
report = classification_report(y_test, y_pred_class64)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     46122
           1       0.58      1.00      0.73     63061

    accuracy                           0.58    109183
   macro avg       0.29      0.50      0.37    109183
weighted avg       0.33      0.58      0.42    109183



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
report = classification_report(y_test, y_pred_class128)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     46122
           1       0.58      1.00      0.73     63061

    accuracy                           0.58    109183
   macro avg       0.79      0.50      0.37    109183
weighted avg       0.76      0.58      0.42    109183



In [25]:
report = classification_report(y_test, y_pred_class256)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     46122
           1       0.58      1.00      0.73     63061

    accuracy                           0.58    109183
   macro avg       0.29      0.50      0.37    109183
weighted avg       0.33      0.58      0.42    109183



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Evaluate

In [27]:
print("Evaluate on test data")
result64_full = model64_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result64_full)

Evaluate on test data


NameError: ignored

In [28]:
print("Evaluate on test data")
result128_full = model128_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result128_full)

Evaluate on test data


NameError: ignored

In [ ]:
print("Evaluate on test data")
result256_full = model256_full.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", result256_full)

Evaluate on test data
853/853 [==============================] - 3s 3ms/step - loss: 0.3192 - acc: 0.8393
test loss, test acc: [0.3191930949687958, 0.8392789959907532]
